## 🧠 Ideal Weight Prediction using Neural Network in TensorFlow
This notebook predicts a person's ideal weight using their height, age, and gender with a neural network built using TensorFlow.

### 🧠 Neural Network Overview
The network consists of 3 hidden layers using ReLU activations and Dropout for regularization.

In [ ]:
# 📦 Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt

import os
if not os.path.exists('ideal_weight_dataset.csv'):
    from google.colab import files
    uploaded = files.upload()

### 📥 Load and Explore the Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['ideal_weight_dataset.csv']))
df.head()


In [ ]:
# ⚙️ Encode gender and scale features
# Normalize column names
df.columns = df.columns.str.strip().str.lower()

# Display columns to verify
print("Columns:", df.columns.tolist())

# Rename columns if necessary
df.rename(columns={
    'height(cm)': 'height',
    'age(years)': 'age',
    'ideal weight': 'ideal_weight'
}, inplace=True)

# Recheck columns
print("Updated Columns:", df.columns.tolist())

# Extract features and encode gender
X = df[['height', 'age', 'gender']].copy()
X['gender'] = LabelEncoder().fit_transform(X['gender'])
y = df['ideal_weight'].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


### 🧠 Build and Train Neural Network

In [ ]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, validation_split=0.1, epochs=150, batch_size=16, verbose=1)

### 📊 Evaluate the Model and Plot Results

In [ ]:
# Evaluate on test data
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {mae:.2f} kg")

In [ ]:
# Plot training history
plt.figure(figsize=(8, 5))
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (kg)')
plt.title('Model Training History')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 💾 Save the trained model
model.save('ideal_weight_neural_net_model.h5')
print("Model saved as 'ideal_weight_neural_net_model.h5'")